In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.anova import AnovaRM
import os
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from scipy.stats import pearsonr, spearmanr

In [91]:
import datetime as dt
from pydriller import Repository
import json
import csv
import time
from tqdm import tqdm


In [92]:
# weird work around to import from helpers folder
# TODO: understand why and how
import sys
os.chdir("C:/Bibliothek/Career/UZH/BA BT Bachelor Arbeit Thesis/Code V2")
sys.path.append(os.getcwd())
from helpers.term_paths import TERM_PATHS
from helpers.term_paths import TERM_SIZES

# Random things

In [93]:
path_df_6 = "C:/Bibliothek/Career/UZH/BA BT Bachelor Arbeit Thesis/Code V2/output/cleaned_aggregated_metrics_v4_061025.csv"

df6 = pd.read_csv(path_df_6) 
df6.shape

(672, 103)

In [94]:
# list all lines that contain total_lines in column names

total_line_cols = [col for col in df6.columns if 'lines of code' in col]
total_line_cols

['lines of code',
 'lines of code tslike',
 'lines of code .ts',
 'lines of code .tsx',
 'lines of code .js',
 'lines of code .jsx']

In [95]:
df6['lines of code tslike']

0       367
1      5604
2      4259
3      3102
4      5879
       ... 
667    5862
668    3627
669    3448
670    3596
671    5331
Name: lines of code tslike, Length: 672, dtype: int64

In [96]:
df6.columns.tolist()

['project index',
 'folder count',
 'file count',
 'file count tslike',
 'file count .ts',
 'file count .tsx',
 'file count .js',
 'file count .jsx',
 'commit count',
 'unique commit authors',
 'commit count per author',
 'first commit date',
 'last commit date',
 'lines of code',
 'lines of code tslike',
 'lines of code .ts',
 'lines of code .tsx',
 'lines of code .js',
 'lines of code .jsx',
 'lines of comments',
 'lines of comments tslike',
 'lines of comments .ts',
 'lines of comments .tsx',
 'lines of comments .js',
 'lines of comments .jsx',
 '/frontend .ts file_count',
 '/frontend .ts total_lines',
 '/frontend .ts sloc',
 '/frontend .ts total_lines_of_comments',
 '/frontend .ts blank_lines',
 '/frontend .js file_count',
 '/frontend .js total_lines',
 '/frontend .js sloc',
 '/frontend .js total_lines_of_comments',
 '/frontend .js blank_lines',
 '/src .ts file_count',
 '/src .ts total_lines',
 '/src .ts sloc',
 '/src .ts total_lines_of_comments',
 '/src .ts blank_lines',
 '/src .j

In [97]:
# sum up lines of code tslike for each term with groupby
df6_term_lines = df6.groupby('term')['3 Folders total_lines'].mean().reset_index()
df6_term_lines


,term,3 Folders total_lines
0,1,3779.375000
1,2,3195.237410
2,3,4062.121547
3,4,5164.655000


In [98]:
df6_term_stats = df6.groupby('term')['3 Folders total_lines'].agg(['min', 'max', 'mean']).reset_index()
df6_term_stats

,term,min,max,mean
0,1,347,14500,3779.375000
1,2,442,9442,3195.237410
2,3,792,13210,4062.121547
3,4,1249,88614,5164.655000


In [99]:
df6_term_stats = df6.groupby('term')['lines of code tslike'].agg(['min', 'max', 'mean']).reset_index()
df6_term_stats

,term,min,max,mean
0,1,367,21023,3797.855263
1,2,553,6783,3147.935252
2,3,953,23781,4155.950276
3,4,1374,114580,5258.185000


In [100]:
df6['lines of code tslike'].sum()

np.int64(2818701)

In [101]:
df6.head()

,project index,folder count,file count,file count tslike,file count .ts,file count .tsx,file count .js,file count .jsx,commit count,unique commit authors,...,TypeScript total_lines_of_comments,TypeScript blank_lines,JavaScript file_count,JavaScript total_lines,JavaScript sloc,JavaScript total_lines_of_comments,JavaScript blank_lines,CC Sum,CC Sum 3folders,term
0,0,31,60,4,3,0,1,0,10,2,...,90,33,0,0,0,0,0,29,29,1
1,1,45,140,44,30,12,2,0,25,3,...,392,469,1,14,14,0,0,694,694,1
2,2,38,249,40,38,0,2,0,155,4,...,712,574,1,116,94,1,20,1262,668,1
3,3,37,201,16,14,0,2,0,270,4,...,156,222,1,272,216,7,42,642,642,1
4,4,44,316,61,46,13,2,0,33,4,...,720,649,1,17,17,0,0,775,756,1


In [102]:
df6.columns[:15]

Index(['project index', 'folder count', 'file count', 'file count tslike',
       'file count .ts', 'file count .tsx', 'file count .js',
       'file count .jsx', 'commit count', 'unique commit authors',
       'commit count per author', 'first commit date', 'last commit date',
       'lines of code', 'lines of code tslike'],
      dtype='object')

In [103]:
df6.columns[15:60]

Index(['lines of code .ts', 'lines of code .tsx', 'lines of code .js',
       'lines of code .jsx', 'lines of comments', 'lines of comments tslike',
       'lines of comments .ts', 'lines of comments .tsx',
       'lines of comments .js', 'lines of comments .jsx',
       '/frontend .ts file_count', '/frontend .ts total_lines',
       '/frontend .ts sloc', '/frontend .ts total_lines_of_comments',
       '/frontend .ts blank_lines', '/frontend .js file_count',
       '/frontend .js total_lines', '/frontend .js sloc',
       '/frontend .js total_lines_of_comments', '/frontend .js blank_lines',
       '/src .ts file_count', '/src .ts total_lines', '/src .ts sloc',
       '/src .ts total_lines_of_comments', '/src .ts blank_lines',
       '/src .js file_count', '/src .js total_lines', '/src .js sloc',
       '/src .js total_lines_of_comments', '/src .js blank_lines',
       '/test/controller .ts file_count', '/test/controller .ts total_lines',
       '/test/controller .ts sloc',
       '/tes

In [104]:
df6.columns[60:]

Index(['/test/rest .js file_count', '/test/rest .js total_lines',
       '/test/rest .js sloc', '/test/rest .js total_lines_of_comments',
       '/test/rest .js blank_lines', '3 Folders file_count',
       '3 Folders total_lines', '3 Folders sloc',
       '3 Folders total_lines_of_comments', '3 Folders blank_lines',
       '/frontend file_count', '/frontend total_lines', '/frontend sloc',
       '/frontend total_lines_of_comments', '/frontend blank_lines',
       '/src file_count', '/src total_lines', '/src sloc',
       '/src total_lines_of_comments', '/src blank_lines',
       '/test/controller file_count', '/test/controller total_lines',
       '/test/controller sloc', '/test/controller total_lines_of_comments',
       '/test/controller blank_lines', '/test/rest file_count',
       '/test/rest total_lines', '/test/rest sloc',
       '/test/rest total_lines_of_comments', '/test/rest blank_lines',
       'TypeScript file_count', 'TypeScript total_lines', 'TypeScript sloc',
       'Typ

In [105]:
df6['/frontend total_lines'].describe()

count      672.000000
mean       739.514881
std       3366.018187
min          0.000000
25%        183.750000
50%        421.500000
75%        794.500000
max      85350.000000
Name: /frontend total_lines, dtype: float64

In [106]:
df6['CC Sum'].describe()

count       672.000000
mean       1338.458333
std        9811.794754
min          29.000000
25%         447.000000
50%         558.000000
75%         724.000000
max      246525.000000
Name: CC Sum, dtype: float64

In [107]:
df6['CC Sum 3folders'].describe()

count       672.000000
mean        978.279762
std        9534.391491
min          29.000000
25%         435.250000
50%         536.000000
75%         663.250000
max      246515.000000
Name: CC Sum 3folders, dtype: float64

# Start

## Plan

Plan
1. Find start dates for each term, maybe even milestone dates
2. Get all commits with pydriller
   1. Create dataframe
   2. Add GenAI Period, Term, Project, User,
   3. Add Start Date
3. Identify and remove bots
   1. ... because that is most likely no human written code.
4. Identify not started / stopped projects
5. Calculate
   1. Time from start date to first commit per project
   2. Size of first commit per project
6. 

| Project     | User        | ...       |
| ----------- | ----------- |-----------|
| 672         | Name        |           |




In [108]:
[
    Project / repository, User, Commit, GenAI, Term, 
Date, Insertions, Deletions, Total lines, Files changed,
Unit size (DMM), Complexity (DMM), Interface (DMM),
Commits, Commit hash, Diff lines,
Copilot period, Merge commit, Default branch, Org,
]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (475053162.py, line 3)

- Org and Commits (=1) not needed
- Add Diff lines 

## Commit Drilling

#### This was the development of commits_py_driller.py file

In [ ]:
# from helpers.term_paths import TERM_PATHS
from pydriller import Repository


def test():
    pass





In [ ]:
def get_term(path: str) -> int | str:
    """Return term number (as integer) based on known term substrings in path."""
    term_map = {"2021W-T1": 1, "2021W-T2": 2, "2023W-T2": 3, "2024W-T1": 4}
    for key, val in term_map.items():
        if key in path:
            return val
    print(f"Path does not match known terms: {path}")
    return "Unknown Term"



def get_genAI_period(path: str) -> bool:
    """Find if repo is in genAI period based on term."""
    term = get_term(path)
    if term in [1, 2]:
        return False
    elif term in [3, 4]:
        return True
    else:
        print("Failed to return genAI period")
        return None

           

In [ ]:
timestr = time.strftime("%d%m%y")
timestr

'231025'

In [ ]:

# adjust these to the data size
TOTAL_PROJECTS_RECEIVED = 681

term_length = {
            1: TERM_SIZES[0],
            2: TERM_SIZES[1],
            3: TERM_SIZES[2],
            4: TERM_SIZES[3]
        }

def process_commits(term_path: str):
    """
    Performs the repository mining and saves data into 
    csv-file.
    Get all commits in all projects for a given term.

    Args:
        term_path (str): path to the term we are mining

    Returns:
        dataframe: dataframe with all projects of the term
        str: filepath to the csv file containing the commit data
    """

    commit_records = []

    # Once per term
    term_number = get_term(term_path)
    genAI_period = get_genAI_period(term_path)
    
    # Define CSV file path
    timestr = time.strftime("%d%m%y")
    output_csv_filename = f"commit_stats_term_{term_number}_{timestr}.csv"
    output_path = "./output/"
    os.makedirs(output_path, exist_ok=True)  # Ensure directory exists
    csv_filepath = os.path.join(output_path, output_csv_filename)

    # Define CSV column headers (in Title Case)
    fieldnames = [
        "GenAI Period", "Term", "Project", 
        "Author Name", "Author Email", "Date", "Repository",
        "Insertions", "Deletions", "Total Lines", "Files Changed",
        "Diff Lines", "File Count",
        "Unit Size (DMM)", "Complexity (DMM)", "Interface (DMM)", 
        "Commit Hash", "Merge Commit", "Default Branch"
    ]

    file_exists = os.path.exists(csv_filepath)

    # Read existing data to show progress
    # TODO: Is this needed?
    if file_exists:
        existing_df = pd.read_csv(csv_filepath)
        if existing_df.empty:
            print("CSV is empty. Starting fresh.")
            print(f"Remaining projects to process: {TOTAL_PROJECTS_RECEIVED}")
        else:
            print(f"CSV contains commit data. {existing_df.shape[0]} commits already processed.")
            print(f"Last project processed: {existing_df['Project'].iloc[-1]}")
            remaining_projects = TOTAL_PROJECTS_RECEIVED - existing_df['Project'].nunique()
            print(f"Remaining projects to process: {remaining_projects}")
    else:
        print("No CSV file found. Starting from scratch.")
        print(f"Remaining projects to process: {TOTAL_PROJECTS_RECEIVED}")

    with open(csv_filepath, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # Write header if the file is newly created
        if not file_exists:
            writer.writeheader()

        project_index = sum(term_length[t] for t in range(1, term_number))

        project_paths_of_term = []
        for d in os.listdir(term_path):
            full_path = os.path.join(term_path, d)
            if os.path.isdir(full_path):
                project_paths_of_term.append(full_path)

        # https://accessibleai.dev/post/extracting-git-data-pydriller/
        for proj_path in project_paths_of_term:
            print(f"Processing project {project_index} of term {term_number}")
            
            # Traversing all commits of a project
            # TODO: Check order of traversing?
            for commit in Repository(path_to_repo=proj_path, histogram_diff=True).traverse_commits():

                # data extracted from each commit
                record = {
                    "GenAI Period": genAI_period,
                    "Term": term_number, 
                    "Project": project_index,

                    "Author Name": commit.author.name.strip(),  
                    "Author Email": commit.author.email,
                    "Date": commit.committer_date.date(),
                    "Repository": commit.project_name,
                    
                    "Insertions": commit.insertions,
                    "Deletions": commit.deletions,
                    "Total Lines": commit.lines,
                    "Files Changed": commit.files,

                    "Diff Lines": commit.insertions - commit.deletions,
                    "File Count": commit.files, # number of files changed

                    "Unit Size (DMM)": commit.dmm_unit_size,
                    "Complexity (DMM)": commit.dmm_unit_complexity,
                    "Interface (DMM)": commit.dmm_unit_interfacing,

                    "Commit Hash": commit.hash,
                    "Merge Commit": commit.merge,
                    "Default Branch": commit.in_main_branch,

                    # 'Message': commit.msg
                }
            
                commit_records.append(record)
                writer.writerow(record)
            
            project_index += 1
        
    # Convert the list of records into a DataFrame and save to CSV
    df = pd.DataFrame(commit_records)
    print(f"***CSV file saved as {output_csv_filename} in {output_path}.***")

    # Return the DataFrame for further analysis if needed
    return df, csv_filepath


def run():
    # One line per term
    # One term at the time
    # process_commits(TERM_PATHS[0])
    # process_commits(TERM_PATHS[1])
    # process_commits(TERM_PATHS[2])
    process_commits(TERM_PATHS[3])
    # print("done")

run()

No CSV file found. Starting from scratch.
Remaining projects to process: 681
Processing project 481 of term 4
Processing project 482 of term 4
Processing project 483 of term 4
Processing project 484 of term 4
Processing project 485 of term 4
Processing project 486 of term 4
Processing project 487 of term 4
Processing project 488 of term 4
Processing project 489 of term 4
Processing project 490 of term 4
Processing project 491 of term 4
Processing project 492 of term 4
Processing project 493 of term 4
Processing project 494 of term 4
Processing project 495 of term 4
Processing project 496 of term 4
Processing project 497 of term 4
Processing project 498 of term 4
Processing project 499 of term 4
Processing project 500 of term 4
Processing project 501 of term 4
Processing project 502 of term 4
Processing project 503 of term 4
Processing project 504 of term 4
Processing project 505 of term 4
Processing project 506 of term 4
Processing project 507 of term 4
Processing project 508 of term 4

#### This was for understanding how to mine commits with pydriller

In [109]:
print("run")

run


In [113]:

# Get all commits with pydriller

p0 = "C:/Bibliothek/Career/UZH/BA BT Bachelor Arbeit Thesis/CPSC310_termData/CPSC310-2021W-T1/project_team037"
p1 = "C:/Bibliothek/Career/UZH/BA BT Bachelor Arbeit Thesis/CPSC310_termData/CPSC310-2021W-T1/project_team038"
p2 = "C:/Bibliothek/Career/UZH/BA BT Bachelor Arbeit Thesis/CPSC310_termData/CPSC310-2021W-T1/project_team064"

def print_commits(path):
    print(f"Commits in repository at {path}:")
    for commit in Repository(path).traverse_commits():
        if commit.msg == "Starter files":
            # print(f"Term: {get_term(path)}")
            # print(f"Project: {counter}")
            # print(f"GenAI Period: {get_genAI_period()}")
            print(f"Name: {commit.author.name}")
            print(f"Mail: {commit.author.email}")
            print(f"Msg:  {commit.msg}")
            print(f"Date: {commit.author_date}")
            print(f"Ins:  {commit.insertions}")
            print(f"Date: {commit.committer_date.date(),}")
            print(f"Repository: {commit.project_name}")
            print(f"Insertions: {commit.insertions}")
            print(f"Deletions: {commit.deletions}")
            print(f"Total lines: {commit.lines}")
            print(f"Files changed: {commit.files}")
            print(f"Unit size (DMM): {commit.dmm_unit_size}")
            print(f"Complexity (DMM): {commit.dmm_unit_complexity}")
            print(f"Interface (DMM): {commit.dmm_unit_interfacing}")
            print(f"Commit hash: {commit.hash}")
            print(f"Merge commit: {commit.merge}")
            print(f"Default branch: {commit.in_main_branch}")
            print(f"committer: {commit.committer}")
            print("-----")

print_commits(p2)

# path parser --> term
# counter for projects in term
# genAI period


def get_unique_author_emails(repo_path):
    """"Based on email, get unique authors"""
    authors = set()
    for commit in Repository(repo_path).traverse_commits():
        authors.add(commit.author.email)
    return authors

def get_unique_author_names(path):
    """"Based on name, get unique authors"""
    authors = set()
    for commit in Repository(path).traverse_commits():
        authors.add(commit.author.name)
    return authors

def get_commit_amount(path):
    return len(list(Repository(path).traverse_commits()))


# Project, User, Commit, GenAI, Term, 
# 1, mail, hash, True, 1
# [0, 681], ..., .... [False, True], [1, 2, 3, 4]

# print(get_unique_author_emails(p2))
# print(get_commit_amount(p0))
# print_commits(p1)
# print(TERM_PATHS[0])

Commits in repository at C:/Bibliothek/Career/UZH/BA BT Bachelor Arbeit Thesis/CPSC310_termData/CPSC310-2021W-T1/project_team064:
Name: classy
Mail: classy@cs.ubc.ca
Msg:  Starter files
Date: 2021-09-21 19:59:16+00:00
Ins:  4696
Date: (datetime.date(2021, 9, 21),)
Repository: project_team064
Insertions: 4696
Deletions: 0
Total lines: 4696
Files changed: 31
Unit size (DMM): 1.0
Complexity (DMM): 1.0
Interface (DMM): 0.9666666666666667
Commit hash: 0a9f9e5cf9ccd6dbab310b4051dbb197e76da208
Merge commit: False
Default branch: True
committer: classy, <classy@cs.ubc.ca>
-----
